# Attention Vizualizer

Howdy! Regardless of what shows up below, click Kernel --> Restart and Run All to get started.

Mouse over each Glimpse to view the post-windowing input to the network. It might be kind of slow, since it's a network round trip, so be patient. Enjoy!

In [1]:
from viz import *

Widget Javascript not detected.  It may not be installed or enabled properly.


W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='36eb976a-4e6e-42b5-b470-36960ed29f16', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='3fe13035-cbb4-4213-bdbd-ea4eb4ca53f7', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='58f4b9e7-63ca-44ae-ba56-7af7800b669d', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='7df8c041-2eb9-4622-8bb1-97ff9c892729', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='b70951d4-0dab-4273-8554-49edcf021086', ...)


INFO:tensorflow:Restoring parameters from model_runs/human_4_eps1/classifymodel_1000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/human_4_eps1/classifymodel_1000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/human_4_eps1/classifymodel_200000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/human_4_eps1/classifymodel_200000.ckpt
